# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HW').getOrCreate()

In [4]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [6]:
path = 'Datasets/uw-madison-courses/'
df_list=[]

for filename in os.listdir(path):
    if filename.endswith('.csv'):
        filename_list = filename.split(".")
        df_name = filename_list[0]
        df = spark.read.csv(path+filename, inferSchema=True, header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name+' = df')

In [7]:
df_list

['courses',
 'course_offerings',
 'grade_distributions',
 'instructors',
 'rooms',
 'schedules',
 'sections',
 'subjects',
 'subject_memberships',
 'teachings']

Now check the contents of a few of the dataframses that were read in above.

In [10]:
def look(df):
    return df.limit(5).toPandas()

look(instructors)

,id,name
0,761703,JOHN ARCHAMBAULT
1,3677061,STEPHANIE KANN
2,788586,KATHY PREM
3,1600463,KRISTIN KLARKOWSKI
4,693634,DAVID BOHNHOFF


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [24]:
df1 = rooms.join(sections, rooms.uuid == sections.room_uuid, how='left').select([
    rooms.uuid,
    rooms.facility_code,
    sections.course_offering_uuid,
    sections.number
])

df2 = df1.join(course_offerings, df1.course_offering_uuid == course_offerings.uuid, how='left').select([
    df1.facility_code,
    df1.number,
    course_offerings.term_code,
    'name'
])

look(df2)

,facility_code,number,term_code,name
0,0140,307,1182,International Business
1,0140,3,1174,Intro Managerial Accounting
2,0140,3,1174,Intro Managerial Accounting
3,0140,311,1174,Business Law
4,0140,309,1182,Business Law


## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [27]:
df2.filter(df2.facility_code == '0469').show(5)

+-------------+------+---------+--------------------+
|facility_code|number|term_code|                name|
+-------------+------+---------+--------------------+
|         0469|   337|     1082|Introduction to P...|
|         0469|   352|     1082|Introduction to P...|
|         0469|   308|     1102|Introduction to P...|
|         0469|   307|     1102|Introduction to P...|
|         0469|   371|     1174|Introduction to P...|
+-------------+------+---------+--------------------+
only showing top 5 rows



## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [35]:
df1 = subjects.join(subject_memberships, subjects.code == subject_memberships.subject_code, how='inner')
df2 = df1.join(sections, df1.course_offering_uuid == sections.course_offering_uuid, how='left')
df3 = df2.join(rooms, df2.room_uuid == rooms.uuid, how='left')

df3 = df3.select('name','facility_code','room_code')
df3.groupBy('name','facility_code').count().show(10)

+--------------------+-------------+-----+
|                name|facility_code|count|
+--------------------+-------------+-----+
|             Zoology|         0060|   36|
|   Political Science|         0557|   10|
|   Computer Sciences|         0402|   69|
|             Physics|         1485|   31|
|Scandinavian Studies|         0046|   62|
|          Physiology|        0451C|   94|
|Biological System...|       ONLINE|    8|
|  Communication Arts|         0084|    2|
|Agricultural and ...|         0055|   45|
|        Soil Science|   OFF CAMPUS|   20|
+--------------------+-------------+-----+
only showing top 10 rows



## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [39]:
look(grade_distributions)

,course_offering_uuid,section_number,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,139,12,2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,075da420-5f49-3dd0-93df-13e3c152e1b1,1,87,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,1,70,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [44]:
df1 = grade_distributions.groupBy('course_offering_uuid').agg(sum('f_count').alias('failed_students'))
look(df1)

,course_offering_uuid,failed_students
0,c939dd5e-43ba-3266-8f74-573f629de75b,0
1,75fdf27b-0e52-3544-96eb-d594a5ad969f,0
2,cfeba94d-8e0e-320b-a904-ea2c2a31c000,7
3,ceabe145-78e5-33c2-9b68-3a1eea9c2864,4
4,1ee15e7e-fa91-3205-8999-4cac58a95b61,0


In [46]:
df2 = df1.join(course_offerings, df1.course_offering_uuid == course_offerings.uuid, how='left')
df2.select(['name','failed_students']).orderBy(desc('failed_students')).show(5, truncate=False)

+----------------------------+---------------+
|name                        |failed_students|
+----------------------------+---------------+
|Calc--Functns of Variables  |72             |
|Animal Biology              |70             |
|Calculus&Analytic Geometry  |67             |
|Calculus&Analytic Geometry 1|64             |
|Calculus&Analytic Geometry  |63             |
+----------------------------+---------------+
only showing top 5 rows



## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [47]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [53]:
options = spark.createDataFrame(['Monkey','Elephant','Hippopotamus'], StringType())
correct_name = zoo.join(options, levenshtein(zoo.Animal, options["value"]) < 5, how='left')
correct_name.show()

+------------+---+------------+
|      Animal|age|       value|
+------------+---+------------+
|      Monkey| 10|      Monkey|
|      Monkay| 36|      Monkey|
|        Mnky|123|      Monkey|
|    Elephant| 48|    Elephant|
|     Elefant| 16|    Elephant|
|    Ellafant|  1|    Elephant|
|Hippopotamus| 48|Hippopotamus|
| Hipopotamus| 16|Hippopotamus|
|       Hippo|  1|        null|
+------------+---+------------+



### Great job! 